In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import numpy as np
from shapely.geometry import shape, mapping
import os

# Function to sample the raster along a geometry
def sample_raster(raster, geom):
    # Transform the geometry into a boolean matrix (mask)
    mask = geometry_mask([geom], out_shape=raster.shape, transform=raster.transform, invert=False)
    # Sample the raster using the mask
    values = raster.read(1, masked=True)
    sampled_values = values[mask]
    return sampled_values

# Function to assign elevations to a shapefile
def assign_elevations(shapefile_path, raster):
    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)
    
    # Sample the raster along the geometries of the shapefile
    elevations = []
    for idx, row in gdf.iterrows():
        geom = row['geometry']
        sampled_values = sample_raster(raster, geom)
        if len(sampled_values) > 0:
            # Calculate the mean of the sampled values
            elevation = np.nanmean(sampled_values)
            elevations.append(elevation)
        else:
            elevations.append(np.nan)

    # Add the elevations to the shapefile dataframe
    gdf['elevation'] = elevations

    # Save the new shapefile with the assigned elevations
    output_shapefile_path = os.path.splitext(shapefile_path)[0] + '_with_elevations.shp'
    gdf.to_file(output_shapefile_path)

    print(f"Elevation assignment for {shapefile_path} completed and file saved successfully.")

# Path to the raster file
rasterfile_path = r"C:\Users\noemi\Dropbox (Politecnico Di Torino Studenti)\Magistrale\II anno_UGent\Thesis\QGIS_PYTHON\DTM+BDTRE_QGIS\DTM\DTM5\DTM5 Cervo Valley.tif"

# Load the raster
raster = rasterio.open(rasterfile_path)

# Path to the folder containing the input shapefiles
input_folder = r"C:\Users\noemi\Dropbox (Politecnico Di Torino Studenti)\Magistrale\II anno_UGent\Thesis\QGIS_PYTHON\DTM+BDTRE_QGIS\Punti di Rilevazione\Pluviometers"

# Process each input file  
for filename in os.listdir(input_folder):
    if filename.endswith('.shp'):
        shapefile_path = os.path.join(input_folder, filename)
        assign_elevations(shapefile_path, raster)

# Close the raster
raster.close()
print("\nElevations assigned correctly and file correctly saved.")

Assegnazione delle elevazioni per C:\Users\noemi\Dropbox (Politecnico Di Torino Studenti)\Magistrale\II anno_UGent\Thesis\QGIS_PYTHON\DTM+BDTRE_QGIS\Punti di Rilevazione\Pluviometers\BIELLA_Pluviometer.shp completata e file salvato correttamente.
Assegnazione delle elevazioni per C:\Users\noemi\Dropbox (Politecnico Di Torino Studenti)\Magistrale\II anno_UGent\Thesis\QGIS_PYTHON\DTM+BDTRE_QGIS\Punti di Rilevazione\Pluviometers\PIEDICAVALLO_Pluviometer.shp completata e file salvato correttamente.

Assegnazione delle elevazioni completata e file salvato correttamente.
